In [6]:
def parse_input(filename):
    with open(filename, 'r') as f:
        data = f.read()
    data = [d.split('\n') for d in data.split('\n\n')]
    tiles_dict = {}
    for square in data:
        ID = square[0].replace('Tile ','').replace(':','')
        top = square[1]
        bottom = square[-1]
        right = ''
        left = ''
        for row in square[1:]:
            left += row[0]
            right += row[-1]
        tiles_dict[ID] = {'left':left, 'top':top, 
                          'right':right, 'bottom':bottom, 
                          'contents':square[1:]}
    return(tiles_dict)

def get_all_edges(squares):
    edges = []
    for square in squares:
        for val in squares[square].values():
            edges.append(val)
    return(edges)

def update_edge_matches(s, edges):
    s['left matches'] = edges.count(s['left']) + edges.count(s['left'][::-1]) - 1
    s['right matches'] = edges.count(s['right']) + edges.count(s['right'][::-1]) - 1
    s['top matches'] = edges.count(s['top']) + edges.count(s['top'][::-1]) - 1
    s['bottom matches'] = edges.count(s['bottom']) + edges.count(s['bottom'][::-1]) - 1
    s['matches'] = s['left matches'] + s['right matches'] + s['top matches'] + s['bottom matches']
        
def find_corners(squares):
    corners = []
    for s in squares:
        if squares[s]['matches'] == 2:
            corners.append(int(s))
    return(corners)

def update_edges(square, edges):
    contents = square['contents']
    square['top'] = contents[0]
    square['bottom'] = contents[-1]
    left = ''
    right = ''
    for row in contents:
        left += row[0]
        right += row[-1]
    square['left'] = left
    square['right'] = right
    update_edge_matches(square, edges)

def flip_horizontal(s, edges):
    s['contents'] = [row[::-1] for row in s['contents']]
    update_edges(s, edges)   
    
def rotate_left(s, edges):
    s['contents'] = [''.join(i) for i in list(zip(*s['contents']))[::-1]]
    update_edges(s, edges)

def get_edge_match(source_id, source_edge, target_dict):
    source = target_dict[source_id][source_edge]
    for ID in target_dict:
        if ID != source_id:
            for e in ['top','bottom','left','right']:
                target = target_dict[ID][e]
                target_reversed = target[::-1]
                if source == target:
                    return(ID, e, 'normal')
                if source == target_reversed:
                    return(ID, e, 'reversed')
    return(None,None,None)
            
def build_row(left_id, tiles_dict, edges):
    match_id = left_id
    row = [match_id]
    while match_id:
        match_id, match_edge, match_type = get_edge_match(match_id, 'right', tiles_dict)
        if match_id: 
            row.append(match_id)
            if match_type == 'normal':
                if match_edge == 'top':
                    flip_horizontal(tiles_dict[match_id], edges)
                    rotate_left(tiles_dict[match_id], edges)
                elif match_edge == 'right':
                    flip_horizontal(tiles_dict[match_id], edges)
                elif match_edge == 'bottom':
                    [rotate_left(tiles_dict[match_id], edges) for _ in range(3)] # rotate 3 times
                elif match_edge == 'left':
                    pass
            elif match_type == 'reversed':
                if match_edge == 'top':
                    rotate_left(tiles_dict[match_id], edges)
                elif match_edge == 'right':
                    [rotate_left(tiles_dict[match_id], edges) for _ in range(2)] # rotate 2 times
                elif match_edge == 'bottom':
                    rotate_left(tiles_dict[match_id], edges)
                    flip_horizontal(tiles_dict[match_id], edges)
                elif match_edge == 'left':
                    flip_horizontal(tiles_dict[match_id], edges)
                    [rotate_left(tiles_dict[match_id], edges) for _ in range(2)] # rotate 2 times
    return(row)

def build_image(matrix, tiles_dict):
    image = []
    for row in matrix:
        image_row = []
        for item in row:
            item_cont = [s[1:-1] for s in tiles_dict[item]['contents'][1:-1]]
            image_row.append(item_cont)
        image.append(image_row)
    item_length = len(tiles_dict[matrix[0][0]]['bottom']) - 2
    grid_length = len(matrix)
    final_image = []
    for j in range(grid_length):
        for i in range(item_length):
            row = ''
            for k in range(grid_length):
                row += image[j][k][i]
            final_image.append(row)
    return(final_image)

def count_monsters(image, monster):
    sm_width = max([dx for dx,dy in monster])
    sm_height = max([dy for dx,dy in monster])
    monster_size = len(monster)
    monsters_found = 0
    
    for y in range(len(image) - sm_height):
        for x in range(len(image[0]) - sm_width):
            correct = 0
            for dx, dy in monster:
                test = image[y + dy][x + dx]
                if test == '#':
                    correct += 1
            if correct == monster_size:
                monsters_found += 1
    return(monsters_found)

def flip_image(image):
    return([row[::-1] for row in image])

def rotate_image(image):
    return([''.join(i) for i in list(zip(*image))[::-1]])   

def solve_problem(input_file):
    # get input data, build dictionary of tiles
    tiles_dict = parse_input(input_file)
    edges = get_all_edges(tiles_dict)
    for s in tiles_dict:
        update_edge_matches(tiles_dict[s], edges)
    corners = find_corners(tiles_dict)

    # find the upper left tile (1 top match, 1 right match)
    for i in range(len(corners)):
        tile = tiles_dict[str(corners[i])]
        if tile['top matches'] == 1 and tile['right matches'] == 1:
            top_left_id = str(corners[i])

    # build the left row (build right, then rotate that row clockwise)
    left_row = build_row(top_left_id, tiles_dict, edges)

    # build out a row for each of thse to make grid of IDs
    final_grid = []
    for ID in left_row:
        [rotate_left(tiles_dict[ID], edges) for _ in range(3)] # 3 counter clockwise = 1 clockwise
        final_grid.append(build_row(ID, tiles_dict, edges))

    # build the image (remove edges and combine into one image)
    image = build_image(final_grid, tiles_dict)

    # coordinates for sea monster
    sea_monster = [(18,0),
                   (0,1),(5,1),(6,1),(11,1),(12,1),(17,1),(18,1),(19,1),
                   (1,2),(4,2),(7,2),(10,2),(13,2),(16,2)]

    # figure out which orientation of image has monsters in it, and count them
    monster_results = []
    for _ in range(2):
        for _ in range(4):
            monster_results.append(count_monsters(image,sea_monster))
            image = rotate_image(image)
        image = flip_image(image)
    possible = sum([row.count('#') for row in image])
    non_monster_squares = possible - (max(monster_results) * len(sea_monster))
    
    result = 1
    for item in corners:
        result *= item
    import numpy as np
    print('Part 1 Answer:',result)
    print('Part 2 Answer:',non_monster_squares)
    
solve_problem('input_day20.txt')

Part 1 Answer: 107399567124539
Part 2 Answer: 1555
